In [1]:
import pandas as pd
import random
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import unicodedata
import inflect
import re
from itertools import chain
#!pip install contractions
import contractions
nltk.download('punkt')



ModuleNotFoundError: No module named 'inflect'

In [3]:
#load in the csv contatining the data
df_original = pd.read_csv('wiki_movie_plots_deduped.csv')

In [5]:
#Function to create dataframes for each genre; using 1000 elements for training 
def dataframe_creator(genre):
    genre = df_original.drop(df_original[df_original['Genre'] != genre].index, inplace = False)
    genre.drop(['Release Year','Origin/Ethnicity','Cast', 'Wiki Page','Director','Release Year'],axis=1, 
               inplace = True)
    genre = genre.sample(frac=1)
    genre = genre.reset_index(drop=True)
    genre_train = genre[0:1000]
    genre_test = genre[1000:len(genre)]
    return (genre_train,genre_test)

In [7]:
#List of Genres we want to run analysis on
df_horror_train,df_horror_test = dataframe_creator('horror')
df_comedy_train,df_comedy_test = dataframe_creator('comedy')

#Creating dataframes for all genres for training
df_train = pd.concat([df_horror_train,df_comedy_train])

#creating dataframes for all genres for testing
df_test = pd.concat([df_horror_test,df_comedy_test])

,Title,Genre,Plot
0,Big Bad Wolf,horror,Two men are hunting in the jungle of Cameroon ...
1,Picture Mommy Dead,horror,"Susan Shelley thinks her father, Edward, kille..."
2,Dr. Jekyll and Mr. Hyde,horror,Henry Jekyll (John Barrymore) is a doctor of m...
3,Countess Dracula,horror,"In 17th-century Hungary, recently widowed Coun..."
4,Dracula,horror,"In Whitby, England in 1913, Count Dracula (Fra..."
...,...,...,...
995,The Shadow of the Cat,horror,"Late at night in early 1900's England, wealthy..."
996,Dead Man's Eyes,horror,Artist Dave Stuart is blinded by a jealous ass...
997,The Slayer,horror,Kay is an abstract visual artist who has been ...
998,The Revenge of Frankenstein,horror,"In 1860, Baron Victor Frankenstein, sentenced ..."


In [ ]:
# Check word type of a word
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
#Function for text pre-processing
def normalizeText(row):
    
    #create a list of punctuations we wish to delete
    punctuations = string.punctuation
    #create a list of stopwords in English
    stopWords = set(stopwords.words('english'))
    
    #change all letters to lower case
    row = row.lower()  
    
    #remove numbers
    row = re.sub(" \d+", " ", row)
                 
    #remove punctutation
    for letter in row: 
    
        if letter in punctuations: 
            row = row.replace(letter, "")
    
    #expand the contraction I'm -> I am
    row = contractions.fix(row)
    # got it from (https://github.com/kootenpv/contractions)
                
    #remove accent char
    row = unicodedata.normalize('NFKD', row).encode('ascii', 'ignore').decode('utf-8', 'ignore')
                 
    #remove extra whitespace convert into a word  
    row = row.strip()
    
    # TOKENIZATION: process of splitting text into smaller piece called tokens.
    tokens = word_tokenize(row)
    
    # lemmatization step played -> play
    lemmatizer = WordNetLemmatizer()
    row = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(row)])
    
    #remove stop words such as "a", "the", "is"
    tokens = word_tokenize(row)
    row = ' '.join([i for i in tokens if not i in stopWords])
     
    return row

In [ ]:
#run the column through the normalizing function
df_train["Plot"] = df_train["Plot"].apply(normalizeText) 

In [ ]:
# Make list from df for "Plot" column
# Param: the df to use
def dfToList(df):
    lst = []
    for i, rows in df.iterrows():
        l = [rows.Plot]
        lst.append(l)
    return list(chain.from_iterable(lst))

# tf-idf vectorisation
# Param: all plots from the df as a list (ie [plot1, plot2,...])
def tfidfVec(plots):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(plots)
    feature_names = vectorizer.get_feature_names()
    dense = tfidf.todense()
    df_tfidf = pd.DataFrame(dense.tolist(), columns=feature_names)
    return df_tfidf

In [ ]:
# test showing how it works on first 5 elements of df_HandC
tfidfVec(dfToList(df_train.head()))
